# Internet Search using Bing API - Bing Chat Clone

In this notebook, we'll delve into the ways in which you can **boost your GPT Smart Search Engine with web search functionalities**, utilizing both Langchain and the Azure Bing Search API service.

As previously discussed in our other notebooks, **harnessing agents and tools is an effective approach**. We aim to leverage the capabilities of OpenAI's large language models (LLM), such as GPT-4 and its successors, to perform the heavy lifting of reasoning and researching on our behalf.

There are numerous instances where it is necessary for our Smart Search Engine to have internet access. For instance, we may wish to **enrich an answer with information available on the web**, or **provide users with up-to-date and recent information**. Regardless of the scenario, we require our engine to base its responses on search results.

By the conclusion of this notebook, you'll have a solid understanding of the **Bing Search API basics**, including **how to create a Web Search Agent using the Bing Search API**, and how these tools can **strengthen your chatbot**. Additionally, you'll learn about **Callback Handlers, their use, and their significance in bot applications**.

In [4]:
import os
import requests
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import MyCustomHandler
from common.prompts import BING_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials2.env")

def printmd(string):
    display(Markdown(string))
    
MODEL_DEPLOYMENT_NAME = "gpt4"

In [12]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://openai-demo-poc-3232.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [19]:
    response = openai.ChatCompletion.create(
                  engine="gpt4",
                  messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": "Who won the world series in 2020?"}
                    ]
                )

    # print the response
    print(response['choices'][0]['message']['content'])

APIConnectionError: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?

## Introduction to Callback Handlers

This following explanation comes directly from the Langchain documentation about Callbacks ([HERE](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.

You can subscribe to these events by using the callbacks argument available throughout the API. This argument is list of handler objects, which are expected to implement one or more of the methods described below in more detail.

**Callback handlers**:<br>
CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

--------------------
We will incorporate a handler for the callbacks, enabling us to observe the response as it streams and to gain insights into the Agent's reasoning process. This will prove incredibly valuable when we aim to stream the bot's responses to users and keep them informed about the ongoing process as they await the answer.

Our custom handler is on the folder `common/callbacks.py`. Go a take a look at it.

In [13]:
cb_handler = MyCustomHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# Now we declare our LLM object with the callback handler 
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=500,
                      streaming=True, callback_manager=cb_manager)

## Creating a custom tool - Bing Search API tool

Langhain has already a pre-created tool called BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py)), however we are going to make it a bit better by using the results function instead of the run function, that way we not only have the text results, but also the title and link(source) of each snippet.

In [14]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("ChatGPTTool does not support async")

Now, we create our REACT agent that uses our custom tool and our custom prompt `BING_PROMPT_PREFIX`

In [15]:
tools = [MyBingSearch(k=5)]
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX})

Try some of the below questions, or others that you might like

In [16]:
QUESTION = "How much is 50 USD in Euros and is it enough for a 5 star hotel in Frankfurt?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "Who won the last superbowl?"
# QUESTION = "What is the deal with Joe Biden's son?"

In [17]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError:

LLM Error: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised APIConnectionError:

LLM Error: Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?


In [11]:
printmd(response)

Error communicating with OpenAI: Invalid URL 'ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview': No scheme supplied. Perhaps you meant https://ENTER YOUR VALUE/openai/deployments/gpt4/chat/completions?api-version=2023-03-15-preview?

In [9]:
# Uncomment if you want to take a look at the custom bing search prompt
# printmd(agent_executor.agent.llm_chain.prompt.template)

# Summary

In this notebook, we learned about Callback Handlers and how to stream the response from the LLM. We also learn how to create a Bing Chat clone using a clever prompt with specific search and formatting instructions.

The result is an agent that can smartly search the web for us and give us the answer to our question with the right url citations and links!

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.